In [ ]:
from dash import Dash, dcc
import dash_ag_grid as dag
import plotly.express as px
import polars as pl
from datetime import datetime
#-------------------------------------------------------------------------------
#  My week 7 submission is an enhancement to the code offered for this exercise.
#  I am still at the Hello World level of dashboard developers.
#-------------------------------------------------------------------------------

# Read the given csv data to polars dataframe df
csv_data_source = 'all_outlays_2025-02-14.csv'
transaction_category = 'IAP - USAID'
df = (
    pl.scan_csv(
        csv_data_source, 
        infer_schema_length=0  # this reads all columns as string
    )
    .select(
        ['transaction_catg_dts', 'transaction_catg_renamed', 'Date', 'Daily']
    )
    .with_columns(
        pl.col('Date').str.strptime(pl.Date, '%Y-%m-%d'),
        pl.col('Daily').cast(pl.Float32)
    )
    .collect()
)
# make df_usaid using filtered, sorted version of big df.
df_usaid = (
    df
    .sort(['transaction_catg_dts', 'Date'], descending=[True, True])
    .filter(pl.col('transaction_catg_renamed') == transaction_category)
    .filter(pl.col('Date') > pl.lit(datetime(2025, 1, 2)))
)

fig = px.line(
    df_usaid, 
    x='Date', 
    y='Daily', 
    markers=True,
    title='Federal expenditures of USAID (billions of dollars)',
    template='simple_white',
    height=500, width=800,
)

app = Dash()
grid = dag.AgGrid(
    rowData=
        df.filter(pl.col('Daily') > 0.0)
        #.select(pl.col('transaction_catg_renamed', 'Date', 'Daily'))
        .to_pandas()
        .to_dict("records"),
    columnDefs=[{"field": i, 'filter': True, 'sortable': True} for i in df_usaid.columns],
    dashGridOptions={"pagination": True}
)

app.layout = [
    grid,
    dcc.Graph(figure=fig),
]


if __name__ == "__main__":
    app.run(debug=True)




In [ ]:
# df

In [ ]:
print(1/0)

In [ ]:

# csv_data_source = 'all_outlays_2025-02-14.csv'
# df = pd.read_csv(csv_data_source)
# df.sort_values(['transaction_catg_dts', 'Date'], ascending=[False, False], inplace=True)

# df_usaid = df.loc[df['transaction_catg_renamed'] == 'IAP - USAID']
# df_usaid['Date'] = pd.to_datetime(df_usaid['Date'])
# df_usaid = df_usaid.loc[df_usaid['Date'] >= '1/2/2025']

# fig = px.line(df_usaid, x='Date', y='Daily', markers=True,
#               title='Federal expenditures of USAID (billions of dollars)')


# app = Dash()
# grid = dag.AgGrid(
#     rowData=df.to_dict("records"),
#     columnDefs=[{"field": i, 'filter': True, 'sortable': True} for i in df.columns],
#     dashGridOptions={"pagination": True}
# )

# app.layout = [
#     grid,
#     dcc.Graph(figure=fig),
# ]


# if __name__ == "__main__":
#     app.run(debug=True)

In [ ]:
print(1/0)

In [ ]:
from dash import Dash, dcc
import dash_ag_grid as dag
import plotly.express as px
import pandas as pd

csv_data_source = 'all_outlays_2025-02-14.csv'
df = pd.read_csv(csv_data_source)
df.sort_values(['transaction_catg_dts', 'Date'], ascending=[False, False], inplace=True)

df_usaid = df.loc[df['transaction_catg_renamed'] == 'IAP - USAID']
df_usaid['Date'] = pd.to_datetime(df_usaid['Date'])
df_usaid = df_usaid.loc[df_usaid['Date'] >= '1/2/2025']

fig = px.line(df_usaid, x='Date', y='Daily', markers=True,
              title='Federal expenditures of USAID (billions of dollars)')


app = Dash()
grid = dag.AgGrid(
    rowData=df.to_dict("records"),
    columnDefs=[{"field": i, 'filter': True, 'sortable': True} for i in df.columns],
    dashGridOptions={"pagination": True}
)

app.layout = [
    grid,
    dcc.Graph(figure=fig),
]


if __name__ == "__main__":
    app.run(debug=True)

In [ ]:
print(1/0)

In [ ]:
import polars as pl
import plotly.express as px
import plotly.graph_objects as go

#-------------------------------------------------------------------------------
#   This code groups NY Times best selling books by publisher, and plots their
#   rank over time. You can see how long they stayed on the top 10 list, and 
#   when they peaked. Only books with 10+ weeks in the top 10 are included
#-------------------------------------------------------------------------------
df = (
    pl.scan_csv('NYT Fiction Bestsellers - Bestsellers.csv')
    .with_columns(pl.col('title').str.to_titlecase())
    .with_columns(
        pl.col('bestsellers_date', 'published_date')
        .str.strptime(pl.Date, '%m/%d/%Y')
    )
    .filter(pl.col('rank') <= 10)  # filter out everything below top 10
    .with_columns(  # title_tot is a count of weeks in the top 10 for each book
        title_tot = pl.col('title').count().over(pl.col('title')),
    )
    # filter out books with less than 10 weeks in the top 10
    .filter(pl.col('title_tot') >= 10)  
    .select(
        pl.col(
            'author', 'title',  'publisher', 'bestsellers_date',
            'rank', 'weeks_on_list'
        )
    )
    .collect()
)

publisher_list = sorted(list(set(df['publisher'])))
for publisher_index, publisher in enumerate(sorted(publisher_list),start=1):
    publisher_titles = (
        df.filter(pl.col('publisher') == publisher)
        .unique('title')
        .sort('title')
        ['title']
        .to_list()
    )
    trace_list = []
    my_colors = px.colors.qualitative.Alphabet
    my_color_count = len(my_colors)
    fig = go.Figure()
    for color_index, title in enumerate(sorted(publisher_titles)):
        df_title = (
            df
            .filter(pl.col('title') == title)
            .select(pl.col('rank', 'bestsellers_date'))
        )
        trace = go.Scatter(
            x=df_title['bestsellers_date'],
            y=df_title['rank'],
            mode='lines+markers',
            line=dict(color=my_colors[color_index % my_color_count]),
            marker=dict(color='gray', size = 5),
            name=title,
            line_shape='spline',
            connectgaps=False
        )
        fig.add_trace(trace)
    fig.update_layout(
        template='simple_white', 
        height=400, width=800,
        yaxis=dict(range=[10, 1]),
        title=f'{publisher_index}) Publisher: {publisher}',
        yaxis_title='New York Times Bestsellers Rank',
        showlegend=True
        )
    fig.update_yaxes(
        range=[10.5, 0.5], # 1 on top, 10 on the bottom
        showgrid=True,
        tickvals=[i for i in range(1, 11)],  # 1 to 10 in steps of 1
        ticktext= [f'#{i}'for i in range(1, 11)],
    )  
    fig.show()
